In [61]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import PyPDFLoader,TextLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from collections import defaultdict
import json
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveJsonSplitter

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_TRACING_V2 = os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_ENDPOINT = os.getenv('LANGCHAIN_ENDPOINT')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
LANGCHAIN_PROJECT = os.getenv('LANGCHAIN_PROJECT')

In [2]:
embedding_function = OpenAIEmbeddings()

# Create the language model
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)

analysis_prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    """
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{context}"
     """,
     ),
    ("human", "{question}")
    ]
)

In [3]:
def csv_files_to_json(file_paths, encoding='cp949'):
    # Initialize a default dictionary to store dictionaries for each ID
    json_data = defaultdict(lambda: defaultdict(list))
    
    # Iterate through each CSV file
    for file_path in file_paths:
        # Extract the file name without extension to use as a key
        file_key = file_path.split('/')[-1].split('.')[0]
        
        # Read the CSV file into a DataFrame with cp949 encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Group the data by 'id'
        for id, group in df.groupby('id'):
            # Convert the group data to a dictionary without the 'id' column
            record = group.drop(columns=['id']).to_dict(orient='records')
            # Append the record under the corresponding file name for the given id
            json_data[str(id)][file_key].extend(record)
    
    # Convert the default dictionary to a regular dictionary and then to a JSON string
    json_str = json.dumps(dict(json_data), indent=4)
    
    return json_str

In [4]:
def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)

In [47]:
# 통합 응답을 생성하는 함수
def DM_reponse(id, question):
    # CSV 파일 경로
    file_paths = ['./data/col_value_change/inbody.csv', './data/col_value_change/patients.csv', './data/col_value_change/vital.csv','./data/col_value_change/reports.csv']
    
       # CSV 파일을 JSON으로 변환하여 문자열 반환
    json_str = csv_files_to_json(file_paths)
    # JSON 문자열을 다시 딕셔너리로 변환
    json_data = json.loads(json_str)
    
    # ID에 해당하는 데이터 가져오기
    inbody_response = json_data.get(str(id), {}).get('inbody', [])
    patients_response = json_data.get(str(id), {}).get('patients', [])
    vital_response = json_data.get(str(id), {}).get('vital', [])
    reports_resonse = json_data.get(str(id), {}).get('reports', [])
    print(reports_resonse)

    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    
    """
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
    )
    loader = PyPDFLoader("./data/당뇨병_가이드라인.pdf")
    
    docs = loader.load_and_split(text_splitter=splitter)
    vectorstore = FAISS.from_documents(docs, embedding_function)
    retriever = vectorstore.as_retriever()

    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        너는 당뇨병 전문 의사야.
        환자의 정보들을 기반으로 당뇨병에 대한 환자의 건강상태에 대해서 말해 가족 중에 당뇨병이 있는 사람도 있고 없는 사람도 있으니 해당 정보도 유심히 봐야 해.
        내가 주는 docs 파일을 기반으로 이 환자에 특화적인 조언도 추가 해
        이를 기반으로 아래처럼 말해라.
        예시) ~ 환자는 가족 중에 당뇨병에 대한 이력이 ~ 하고 현재 환자는 당뇨병이 ~ 한 상태이다.
        context: {context}
        """       
        ),
        ("human", "{total_analysis}")
    ])

    final_chain = (
        {
            "context": retriever | RunnableLambda(format_docs),
            "total_analysis": RunnablePassthrough(),
        }
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(total_analysis)
    return response

In [48]:
print(DM_reponse(6,"이 환자의 당뇨여부 알려줘"))

[{'측정일자': '2024-05-03', '환자부모_뇌졸중': '없음', '환자부모_심근경색': '없음', '환자부모_고혈압': '없음', '환자부모_당뇨병': '없음', '환자부모_기타(암_포함)': '없음', '환자_뇌졸중': '없음', '환자_심근경색': '있음', '환자_고혈압': '있음', '환자_당뇨병': '있음', '환자_COVID-19(감염횟수)': 0, '환자_기타(암_포함)': '없음', '실내운동': '안함', '실외운동': '함', '평소_즐겨하는_운동_종목': '헬스', '숨_많이_차는_고강도_운동_시행하는_일주일_기준_일수': '0일', '평균_하루_운동시간': '60분', '최근 1주일 동안 근력 운동한 일수': '3일', '흡연여부': '아니오', '과거_흡연_년수': 0, '금연_전_주간_흡연일수': '0일', '현재_흡연_년수': 0, '현재_흡연중이며_한_주_흡연량': '0일', '전자담배_사용여부': '사용한 적 없음', '전자담배_사용경험이_있으며_최근_한달_내_전자담배_사용여부': '사용한 적 없음', '가슴과_관련된_증상': '가슴이 답답합니다', '근력과_관련된_증상': '근력이 감소하였습니다', '기침과_관련된_증상': '해당 사항 없음', '두통과_관련된_증상': '해당 사항 없음', '소변과_관련된_증상': '평소에 소변을 자주 봅니다, 야간에 소변을 자주 봅니다', '식욕과_관련된_증상': '식욕이 없습니다', '심장과_관련된_증상': '가슴이 두근거립니다', '의식과_관련된_증상': '해당 사항 없음', '체중과_관련된_증상': '체중이 증가하였습니다', '열과_관련된_증상': '열이 조금 납니다', '피로감과_관련된_증상': '해당 사항 없음', '피부모양과_관련된_증상': '해당 사항 없음', '피부색과_관련된_증상': '해당 사항 없음', '호흡과_관련된_증상': '해당 사항 없음', '지난_1년간_음주_빈도': 'week 기준 2번', '지난_1년간_평균_음주량': '소주6잔', '지난_1년간_최대_

In [51]:
# 통합 응답을 생성하는 함수
def HBP_reponse(id, question):
    # CSV 파일 경로
    file_paths = ['./data/col_value_change/inbody.csv', './data/col_value_change/patients.csv', './data/col_value_change/vital.csv','./data/col_value_change/reports.csv']
    
       # CSV 파일을 JSON으로 변환하여 문자열 반환
    json_str = csv_files_to_json(file_paths)
    # JSON 문자열을 다시 딕셔너리로 변환
    json_data = json.loads(json_str)
    
    # ID에 해당하는 데이터 가져오기
    inbody_response = json_data.get(str(id), {}).get('inbody', [])
    patients_response = json_data.get(str(id), {}).get('patients', [])
    vital_response = json_data.get(str(id), {}).get('vital', [])
    reports_resonse = json_data.get(str(id), {}).get('reports', [])
    # print(reports_resonse)
    
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    
    """
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
    )
    loader = PyPDFLoader("./data/고혈압_가이드라인.pdf")
    
    docs = loader.load_and_split(text_splitter=splitter)
    vectorstore = FAISS.from_documents(docs, embedding_function)
    retriever = vectorstore.as_retriever()

    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        너는 고혈압 전문 의사야.
        환자의 정보들을 기반으로 고혈압에 대한 환자의 건강상태에 대해서 말해 가족 중에 고혈압이 있는 사람도 있고 없는 사람도 있으니 해당 정보도 유심히 봐야 해.
        내가 주는 docs 파일을 기반으로 이 환자에 특화적인 조언도 추가 해
        이를 기반으로 아래처럼 말해라.
        예시) ~ 환자는 가족 중에 고혈압에 대한 이력이 ~ 하고 현재 환자는 고혈압이 ~ 한 상태이다.
        context: {context}
        """       
        ),
        ("human", "{total_analysis}")
    ])

    final_chain = (
        {
            "context": retriever | RunnableLambda(format_docs),
            "total_analysis": RunnablePassthrough(),
        }
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(total_analysis)
    return response

In [52]:
print(HBP_reponse(6,"이 환자의 고혈압여부 알려줘"))

[{'측정일자': '2024-05-03', '환자부모_뇌졸중': '없음', '환자부모_심근경색': '없음', '환자부모_고혈압': '없음', '환자부모_당뇨병': '없음', '환자부모_기타(암_포함)': '없음', '환자_뇌졸중': '없음', '환자_심근경색': '있음', '환자_고혈압': '있음', '환자_당뇨병': '있음', '환자_COVID-19(감염횟수)': 0, '환자_기타(암_포함)': '없음', '실내운동': '안함', '실외운동': '함', '평소_즐겨하는_운동_종목': '헬스', '숨_많이_차는_고강도_운동_시행하는_일주일_기준_일수': '0일', '평균_하루_운동시간': '60분', '최근 1주일 동안 근력 운동한 일수': '3일', '흡연여부': '아니오', '과거_흡연_년수': 0, '금연_전_주간_흡연일수': '0일', '현재_흡연_년수': 0, '현재_흡연중이며_한_주_흡연량': '0일', '전자담배_사용여부': '사용한 적 없음', '전자담배_사용경험이_있으며_최근_한달_내_전자담배_사용여부': '사용한 적 없음', '가슴과_관련된_증상': '가슴이 답답합니다', '근력과_관련된_증상': '근력이 감소하였습니다', '기침과_관련된_증상': '해당 사항 없음', '두통과_관련된_증상': '해당 사항 없음', '소변과_관련된_증상': '평소에 소변을 자주 봅니다, 야간에 소변을 자주 봅니다', '식욕과_관련된_증상': '식욕이 없습니다', '심장과_관련된_증상': '가슴이 두근거립니다', '의식과_관련된_증상': '해당 사항 없음', '체중과_관련된_증상': '체중이 증가하였습니다', '열과_관련된_증상': '열이 조금 납니다', '피로감과_관련된_증상': '해당 사항 없음', '피부모양과_관련된_증상': '해당 사항 없음', '피부색과_관련된_증상': '해당 사항 없음', '호흡과_관련된_증상': '해당 사항 없음', '지난_1년간_음주_빈도': 'week 기준 2번', '지난_1년간_평균_음주량': '소주6잔', '지난_1년간_최대_

In [59]:
# 통합 응답을 생성하는 함수
def HD_reponse(id, question):
    # CSV 파일 경로
    file_paths = ['./data/col_value_change/inbody.csv', './data/col_value_change/patients.csv', './data/col_value_change/vital.csv','./data/col_value_change/reports.csv']
    
       # CSV 파일을 JSON으로 변환하여 문자열 반환
    json_str = csv_files_to_json(file_paths)
    # JSON 문자열을 다시 딕셔너리로 변환
    json_data = json.loads(json_str)
    
    # ID에 해당하는 데이터 가져오기
    inbody_response = json_data.get(str(id), {}).get('inbody', [])
    patients_response = json_data.get(str(id), {}).get('patients', [])
    vital_response = json_data.get(str(id), {}).get('vital', [])
    reports_resonse = json_data.get(str(id), {}).get('reports', [])
    # print(reports_resonse)
    
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    
    """
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
    )
    # loader = PyPDFLoader("./data/심부전_가이드라인.pdf")
    # 
    # docs = loader.load_and_split(text_splitter=splitter)
      # 첫 번째 PDF 파일 로드
    loader1 = PyPDFLoader("./data/심부전_가이드라인.pdf")
    docs1 = loader1.load_and_split(text_splitter=splitter)
    
    # 두 번째 PDF 파일 로드
    loader2 = PyPDFLoader("./data/심장질환_가이드라인.pdf")
    docs2 = loader2.load_and_split(text_splitter=splitter)
    
    # 두 개의 문서를 합치기
    docs = docs1 + docs2
    vectorstore = FAISS.from_documents(docs, embedding_function)
    retriever = vectorstore.as_retriever()

    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        너는 심장질환 전문 의사야.
        환자의 정보들을 기반으로 심장질환에 대한 환자의 건강상태에 대해서 말해 가족 중에 심장질환이 있는 사람도 있고 없는 사람도 있으니 해당 정보도 유심히 봐야 해.
        내가 주는 docs 파일을 기반으로 이 환자에 특화적인 조언도 추가 해
        이를 기반으로 아래처럼 말해라.
        예시) ~ 환자는 가족 중에 심장질환에 대한 이력이 ~ 하고 현재 환자는 심장질환이 ~ 한 상태이다.
        context: {context}
        """       
        ),
        ("human", "{total_analysis}")
    ])

    final_chain = (
        {
            "context": retriever | RunnableLambda(format_docs),
            "total_analysis": RunnablePassthrough(),
        }
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(total_analysis)
    return response

In [60]:
print(HD_reponse(6,"이 환자의 심장질환여부 알려줘"))

[{'측정일자': '2024-05-03', '환자부모_뇌졸중': '없음', '환자부모_심근경색': '없음', '환자부모_고혈압': '없음', '환자부모_당뇨병': '없음', '환자부모_기타(암_포함)': '없음', '환자_뇌졸중': '없음', '환자_심근경색': '있음', '환자_고혈압': '있음', '환자_당뇨병': '있음', '환자_COVID-19(감염횟수)': 0, '환자_기타(암_포함)': '없음', '실내운동': '안함', '실외운동': '함', '평소_즐겨하는_운동_종목': '헬스', '숨_많이_차는_고강도_운동_시행하는_일주일_기준_일수': '0일', '평균_하루_운동시간': '60분', '최근 1주일 동안 근력 운동한 일수': '3일', '흡연여부': '아니오', '과거_흡연_년수': 0, '금연_전_주간_흡연일수': '0일', '현재_흡연_년수': 0, '현재_흡연중이며_한_주_흡연량': '0일', '전자담배_사용여부': '사용한 적 없음', '전자담배_사용경험이_있으며_최근_한달_내_전자담배_사용여부': '사용한 적 없음', '가슴과_관련된_증상': '가슴이 답답합니다', '근력과_관련된_증상': '근력이 감소하였습니다', '기침과_관련된_증상': '해당 사항 없음', '두통과_관련된_증상': '해당 사항 없음', '소변과_관련된_증상': '평소에 소변을 자주 봅니다, 야간에 소변을 자주 봅니다', '식욕과_관련된_증상': '식욕이 없습니다', '심장과_관련된_증상': '가슴이 두근거립니다', '의식과_관련된_증상': '해당 사항 없음', '체중과_관련된_증상': '체중이 증가하였습니다', '열과_관련된_증상': '열이 조금 납니다', '피로감과_관련된_증상': '해당 사항 없음', '피부모양과_관련된_증상': '해당 사항 없음', '피부색과_관련된_증상': '해당 사항 없음', '호흡과_관련된_증상': '해당 사항 없음', '지난_1년간_음주_빈도': 'week 기준 2번', '지난_1년간_평균_음주량': '소주6잔', '지난_1년간_최대_

In [74]:
# 통합 응답을 생성하는 함수
def cancer_reponse(id, question):
    # CSV 파일 경로
    file_paths = ['./data/col_value_change/inbody.csv', './data/col_value_change/patients.csv', './data/col_value_change/vital.csv','./data/col_value_change/reports.csv']
    
       # CSV 파일을 JSON으로 변환하여 문자열 반환
    json_str = csv_files_to_json(file_paths)
    # JSON 문자열을 다시 딕셔너리로 변환
    json_data = json.loads(json_str)
    
    # ID에 해당하는 데이터 가져오기
    inbody_response = json_data.get(str(id), {}).get('inbody', [])
    patients_response = json_data.get(str(id), {}).get('patients', [])
    vital_response = json_data.get(str(id), {}).get('vital', [])
    reports_resonse = json_data.get(str(id), {}).get('reports', [])
    # print(reports_resonse)
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    
    """
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
    )
    loader = TextLoader("./data/암_지침서.txt")
    
    docs = loader.load_and_split(text_splitter=splitter)
    vectorstore = FAISS.from_documents(docs, embedding_function)
    retriever = vectorstore.as_retriever()

    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        너는 암 전문 의사야.
        환자의 정보들을 기반으로 암에 대한 환자의 건강상태에 대해서 말해 가족 중에 암이 있는 사람도 있고 없는 사람도 있으니 해당 정보도 유심히 봐야 해.
        내가 주는 docs 파일을 기반으로 이 환자에 특화적인 조언도 추가 해.
        암 관련 얘기만 해라.
        이를 기반으로 아래처럼 말해라.
        예시) ~ 환자는 가족 중에 암에 대한 이력이 ~ 하고 현재 환자는 암이 ~ 한 상태이다.
        context: {context}
        """       
        ),
        ("human", "{total_analysis}")
    ])

    final_chain = (
        {
            "context": retriever | RunnableLambda(format_docs),
            "total_analysis": RunnablePassthrough(),
        }
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(total_analysis)
    return response

In [75]:
print(cancer_reponse(6,"이 환자의 암여부 알려줘"))

[{'측정일자': '2024-05-03', '환자부모_뇌졸중': '없음', '환자부모_심근경색': '없음', '환자부모_고혈압': '없음', '환자부모_당뇨병': '없음', '환자부모_기타(암_포함)': '없음', '환자_뇌졸중': '없음', '환자_심근경색': '있음', '환자_고혈압': '있음', '환자_당뇨병': '있음', '환자_COVID-19(감염횟수)': 0, '환자_기타(암_포함)': '없음', '실내운동': '안함', '실외운동': '함', '평소_즐겨하는_운동_종목': '헬스', '숨_많이_차는_고강도_운동_시행하는_일주일_기준_일수': '0일', '평균_하루_운동시간': '60분', '최근 1주일 동안 근력 운동한 일수': '3일', '흡연여부': '아니오', '과거_흡연_년수': 0, '금연_전_주간_흡연일수': '0일', '현재_흡연_년수': 0, '현재_흡연중이며_한_주_흡연량': '0일', '전자담배_사용여부': '사용한 적 없음', '전자담배_사용경험이_있으며_최근_한달_내_전자담배_사용여부': '사용한 적 없음', '가슴과_관련된_증상': '가슴이 답답합니다', '근력과_관련된_증상': '근력이 감소하였습니다', '기침과_관련된_증상': '해당 사항 없음', '두통과_관련된_증상': '해당 사항 없음', '소변과_관련된_증상': '평소에 소변을 자주 봅니다, 야간에 소변을 자주 봅니다', '식욕과_관련된_증상': '식욕이 없습니다', '심장과_관련된_증상': '가슴이 두근거립니다', '의식과_관련된_증상': '해당 사항 없음', '체중과_관련된_증상': '체중이 증가하였습니다', '열과_관련된_증상': '열이 조금 납니다', '피로감과_관련된_증상': '해당 사항 없음', '피부모양과_관련된_증상': '해당 사항 없음', '피부색과_관련된_증상': '해당 사항 없음', '호흡과_관련된_증상': '해당 사항 없음', '지난_1년간_음주_빈도': 'week 기준 2번', '지난_1년간_평균_음주량': '소주6잔', '지난_1년간_최대_

In [76]:
def final_response(id, question):
    DM_response = DM_reponse(id, question)
    HBP_response = HBP_reponse(id, question)
    HD_response = HD_reponse(id, question)
    cancer_resonse = cancer_reponse(id, question)
    total_analysis = f"""
    당뇨병 분석 :
    {DM_response}
    
    고혈압 분석 : 
    {HBP_response}
    
    심장질환 분석 : 
    {HD_response}
    
    암 분석 :
    {cancer_resonse} 
    """
    
    # final_query = f"""
    # You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information. :\n\n{total_analysis}"""
    final_query = f"""
    You are a professional doctor. We provide detailed analysis of the patient's condition and comprehensive advice based on the following context: Please include only the “Summary Analysis” and “Comprehensive” sections in your response. In addition, if the patient or family member has diabetes, high blood pressure, heart disease, or cancer, they will be given a detailed explanation of the disease. If none of the diseases described above apply, measures should be taken to prepare for or prevent such diseases based on comprehensive health status information. Include advice on how to get drunk. Do not include a title or introductory/concluding sentences. Answer the questions using only the context provided. If you don't know the answer, just say you don't know. Do not guess or make up information. :\n\n{total_analysis}"""

    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{question}")
    ])
    final_chain = (
        {"question": RunnablePassthrough()}
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(question)
    return response

In [77]:
print(final_response('6','건강정보에 대한 종합적인 소견을 말해'))

[{'측정일자': '2024-05-03', '환자부모_뇌졸중': '없음', '환자부모_심근경색': '없음', '환자부모_고혈압': '없음', '환자부모_당뇨병': '없음', '환자부모_기타(암_포함)': '없음', '환자_뇌졸중': '없음', '환자_심근경색': '있음', '환자_고혈압': '있음', '환자_당뇨병': '있음', '환자_COVID-19(감염횟수)': 0, '환자_기타(암_포함)': '없음', '실내운동': '안함', '실외운동': '함', '평소_즐겨하는_운동_종목': '헬스', '숨_많이_차는_고강도_운동_시행하는_일주일_기준_일수': '0일', '평균_하루_운동시간': '60분', '최근 1주일 동안 근력 운동한 일수': '3일', '흡연여부': '아니오', '과거_흡연_년수': 0, '금연_전_주간_흡연일수': '0일', '현재_흡연_년수': 0, '현재_흡연중이며_한_주_흡연량': '0일', '전자담배_사용여부': '사용한 적 없음', '전자담배_사용경험이_있으며_최근_한달_내_전자담배_사용여부': '사용한 적 없음', '가슴과_관련된_증상': '가슴이 답답합니다', '근력과_관련된_증상': '근력이 감소하였습니다', '기침과_관련된_증상': '해당 사항 없음', '두통과_관련된_증상': '해당 사항 없음', '소변과_관련된_증상': '평소에 소변을 자주 봅니다, 야간에 소변을 자주 봅니다', '식욕과_관련된_증상': '식욕이 없습니다', '심장과_관련된_증상': '가슴이 두근거립니다', '의식과_관련된_증상': '해당 사항 없음', '체중과_관련된_증상': '체중이 증가하였습니다', '열과_관련된_증상': '열이 조금 납니다', '피로감과_관련된_증상': '해당 사항 없음', '피부모양과_관련된_증상': '해당 사항 없음', '피부색과_관련된_증상': '해당 사항 없음', '호흡과_관련된_증상': '해당 사항 없음', '지난_1년간_음주_빈도': 'week 기준 2번', '지난_1년간_평균_음주량': '소주6잔', '지난_1년간_최대_